In [2]:
import pandas as pd
import numpy as np
import sklearn

In [3]:
print('pandas',np.array(pd.__version__))
print('nump',np.array(np.__version__))
print('sklearn',np.array(sklearn.__version__))

pandas 2.1.4
nump 1.26.4
sklearn 1.3.2


In [5]:
#Load movies data
movies=pd.read_csv("/content/drive/MyDrive/tmdb_5000_movies.csv")

#load credit data
credit=pd.read_csv("/content/drive/MyDrive/tmdb_5000_credits.csv")


#join two dataframe based on title
movies=movies.merge(credit,on='title')

#count the langage based
movies['original_language'].value_counts()

#select the specific column
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

#check null data
movies.isnull().sum()


#drop null data
movies.dropna(inplace=True)

#check again
movies.isnull().sum()

#check duplicate data
movies.duplicated().sum()

movies.iloc[0].genres


#change generes column into array format
import ast      #for change to string import library
def convert(obj):
    lst=[]
    for i in ast.literal_eval(obj):
        lst.append(i['name'])
    return lst

movies['genres']=movies['genres'].apply(convert)

#change keyword column into array format
movies['keywords']=movies['keywords'].apply(convert)

#select top3 cast name in cast column
def convert_cast(obj):
    lst=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:
            lst.append(i['name'])
            counter+=1
        else:
            break
    return lst

movies['cast']=movies['cast'].apply(convert_cast)

#crew column through get director name
def fetch_director(obj):
    lst=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            lst.append(i['name'])
            break
    return lst

movies['crew']=movies['crew'].apply(fetch_director)

#overview column to split with space and make word
movies['overview']=movies['overview'].apply(lambda x:x.split())

#transform data between spaces
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])

#create tags column from oveview,keywors,genres,cast,crew column
movies['tags']=movies['overview']+movies['keywords']+movies['genres']+movies['cast']+movies['crew']

#take only get required column
new_df=movies[['movie_id','title','tags']]

#convert overview column to list to string
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

#make lowecase of overview column
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

#Machine Learning
#Make vectorization of tags column
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

#avoid same words like going,gone,go
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

new_df['tags']=new_df['tags'].apply(stem)

#create vector
vectors=cv.fit_transform(new_df['tags']).toarray()

#take words from corpus
cv.get_feature_names_out()

#similartity find cosine Simliraty
from sklearn.metrics.pairwise import cosine_similarity

#find distance
similarity=cosine_similarity(vectors)

#recomedantation
def recomend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances =similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)



<ipython-input-5-77ce72dca854>:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))
<ipython-input-5-77ce72dca854>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())
<ipython-input-5-77ce72dca854>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [9]:
import pickle

In [8]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [11]:
pickle.dump(similarity,open('similarity.pkl','wb'))